# gpt-prompt-engineer
By Matt Shumer (https://twitter.com/mattshumer_)

Github repo: https://github.com/mshumer/gpt-prompt-engineer

Generate an optimal prompt for a given task.

To generate a prompt:
1. In the first cell, add in your OpenAI key.
2. If you don't have GPT-4 access, change `model='gpt-4'` in the second cell to `model='gpt-3.5-turbo'`. If you do have access, skip this step.
2. In the last cell, fill in the description of your task, up to 15 test cases, and the number of prompts to generate.
3. Run all the cells! The AI will generate a number of candidate prompts, and test them all to find the best one!

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Experiment')
os.chdir('PromptEngineering')
os.chdir('gpt-prompt-engineer')

In [3]:
!pip install openai prettytable tqdm tenacity wandb -qq
!pip install python-dotenv

In [4]:
from prettytable import PrettyTable
import time
import openai
from tqdm import tqdm
import itertools
import wandb
from tenacity import retry, stop_after_attempt, wait_exponential


use_wandb = False # set to True if you want to use wandb to log your config and results

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [6]:
system_gen_system_prompt = """Your job is to generate system prompts for GPT-4, given a description of the use-case and some test cases.

The prompts you will be generating will be for freeform tasks, such as generating a landing page headline, an intro paragraph, solving a math problem, etc.

In your generated prompt, you should describe how the AI should behave in plain English. Include what it will see, and what it's allowed to output. Be creative with prompts to get the best possible results. The AI knows it's an AI -- you don't need to tell it this.

You will be graded based on the performance of your prompt... but don't cheat! You cannot include specifics about the test cases in your prompt. Any prompts with examples will be disqualified.

Most importantly, output NOTHING but the prompt. Do not include anything else in your message."""


ranking_system_prompt = """Your job is to rank the quality of two outputs generated by different prompts. The prompts are used to generate a response for a given task.

You will be provided with the task description, the test prompt, and two generations - one for each system prompt.

Rank the generations in order of quality. If Generation A is better, respond with 'A'. If Generation B is better, respond with 'B'.

Remember, to be considered 'better', a generation must not just be good, it must be noticeably superior to the other.

Also, keep in mind that you are a very harsh critic. Only rank a generation as better if it truly impresses you more than the other.

Respond with your ranking, and nothing else. Be fair and unbiased in your judgement."""

In [7]:
# K is a constant factor that determines how much ratings change
K = 32

CANDIDATE_MODEL = 'gpt-3.5-turbo' # change this to gpt-3.5-turbo if you don't have GPT-4 access
CANDIDATE_MODEL_TEMPERATURE = 0.9

GENERATION_MODEL = 'gpt-3.5-turbo'
GENERATION_MODEL_TEMPERATURE = 0.8
GENERATION_MODEL_MAX_TOKENS = 60

N_RETRIES = 3  # number of times to retry a call to the ranking model if it fails
RANKING_MODEL = 'gpt-3.5-turbo'
RANKING_MODEL_TEMPERATURE = 0.5

NUMBER_OF_PROMPTS = 10 # this determines how many candidate prompts to generate... the higher, the more expensive, but the better the results will be

WANDB_PROJECT_NAME = "gpt-prompt-eng" # used if use_wandb is True, Weights &| Biases project name
WANDB_RUN_NAME = None # used if use_wandb is True, optionally set the Weights & Biases run name to identify this run

In [8]:
def start_wandb_run():
    # start a new wandb run and log the config
    wandb.init(
      project=WANDB_PROJECT_NAME,
      name=WANDB_RUN_NAME,
      config={
        "K": K,
        "system_gen_system_prompt": system_gen_system_prompt,
        "ranking_system_prompt": ranking_system_prompt,
        "candiate_model": CANDIDATE_MODEL,
        "candidate_model_temperature": CANDIDATE_MODEL_TEMPERATURE,
        "generation_model": GENERATION_MODEL,
        "generation_model_temperature": GENERATION_MODEL_TEMPERATURE,
        "generation_model_max_tokens": GENERATION_MODEL_MAX_TOKENS,
        "n_retries": N_RETRIES,
        "ranking_model": RANKING_MODEL,
        "ranking_model_temperature": RANKING_MODEL_TEMPERATURE,
        "number_of_prompts": NUMBER_OF_PROMPTS
        })

    return

In [9]:
# Optional logging to Weights & Biases to reocrd the configs, prompts and results
if use_wandb:
    start_wandb_run()

In [10]:
def generate_candidate_prompts(description, test_cases, number_of_prompts):
    outputs = openai.ChatCompletion.create(
        model=CANDIDATE_MODEL,
        messages=[
            {"role": "system", "content": system_gen_system_prompt},
            {"role": "user", "content": f"Here are some test cases:`{test_cases}`\n\nHere is the description of the use-case: `{description.strip()}`\n\nRespond with your prompt, and nothing else. Be creative."}
            ],
        temperature=CANDIDATE_MODEL_TEMPERATURE,
        n=number_of_prompts)

    prompts = []

    for i in outputs.choices:
        prompts.append(i.message.content)
    return prompts

def expected_score(r1, r2):
    return 1 / (1 + 10**((r2 - r1) / 400))

def update_elo(r1, r2, score1):
    e1 = expected_score(r1, r2)
    e2 = expected_score(r2, r1)
    return r1 + K * (score1 - e1), r2 + K * ((1 - score1) - e2)

# Get Score - retry up to N_RETRIES times, waiting exponentially between retries.
@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_score(description, test_case, pos1, pos2, ranking_model_name, ranking_model_temperature):
    score = openai.ChatCompletion.create(
        model=ranking_model_name,
        messages=[
            {"role": "system", "content": ranking_system_prompt},
            {"role": "user", "content": f"""Task: {description.strip()}
Prompt: {test_case['prompt']}
Generation A: {pos1}
Generation B: {pos2}"""}
        ],
        logit_bias={
              '32': 100,  # 'A' token
              '33': 100,  # 'B' token
        },
        max_tokens=1,
        temperature=ranking_model_temperature,
    ).choices[0].message.content
    return score

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_generation(prompt, test_case):
    generation = openai.ChatCompletion.create(
        model=GENERATION_MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"{test_case['prompt']}"}
        ],
        max_tokens=GENERATION_MODEL_MAX_TOKENS,
        temperature=GENERATION_MODEL_TEMPERATURE,
    ).choices[0].message.content
    return generation

def test_candidate_prompts(test_cases, description, prompts):
    # Initialize each prompt with an ELO rating of 1200
    prompt_ratings = {prompt: 1200 for prompt in prompts}

    # Calculate total rounds for progress bar
    total_rounds = len(test_cases) * len(prompts) * (len(prompts) - 1) // 2

    # Initialize progress bar
    pbar = tqdm(total=total_rounds, ncols=70)

    # For each pair of prompts
    for prompt1, prompt2 in itertools.combinations(prompts, 2):
        # For each test case
        for test_case in test_cases:
            # Update progress bar
            pbar.update()

            # Generate outputs for each prompt
            generation1 = get_generation(prompt1, test_case)
            generation2 = get_generation(prompt2, test_case)

            # Rank the outputs
            score1 = get_score(description, test_case, generation1, generation2, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)
            score2 = get_score(description, test_case, generation2, generation1, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)

            # Convert scores to numeric values
            score1 = 1 if score1 == 'A' else 0 if score1 == 'B' else 0.5
            score2 = 1 if score2 == 'B' else 0 if score2 == 'A' else 0.5

            # Average the scores
            score = (score1 + score2) / 2

            # Update ELO ratings
            r1, r2 = prompt_ratings[prompt1], prompt_ratings[prompt2]
            r1, r2 = update_elo(r1, r2, score)
            prompt_ratings[prompt1], prompt_ratings[prompt2] = r1, r2

            # Print the winner of this round
            if score > 0.5:
                print(f"Winner: {prompt1}")
            elif score < 0.5:
                print(f"Winner: {prompt2}")
            else:
                print("Draw")

    # Close progress bar
    pbar.close()

    return prompt_ratings


def generate_optimal_prompt(description, test_cases, number_of_prompts=10, use_wandb=False):
    if use_wandb:
        wandb_table = wandb.Table(columns=["Prompt", "Ranking"])
        if wandb.run is None:
            start_wandb_run()

    prompts = generate_candidate_prompts(description, test_cases, number_of_prompts)
    prompt_ratings = test_candidate_prompts(test_cases, description, prompts)

    # Print the final ELO ratingsz
    table = PrettyTable()
    table.field_names = ["Prompt", "Rating"]
    for prompt, rating in sorted(prompt_ratings.items(), key=lambda item: item[1], reverse=True):
        table.add_row([prompt, rating])
        if use_wandb:
            wandb_table.add_data(prompt, rating)

    if use_wandb: # log the results to a Weights & Biases table and finsih the run
        wandb.log({"prompt_ratings": wandb_table})
        wandb.finish()
    print(table)

# In the cell below, fill in your description and test cases

In [11]:
description = "Given a prompt, generate a landing page headline." # this style of description tends to work well

test_cases = [
    {
        'prompt': 'Promoting an innovative new fitness app, Smartly',
    },
    {
        'prompt': 'Why a vegan diet is beneficial for your health',
    },
    {
        'prompt': 'Introducing a new online course on digital marketing',
    },
    {
        'prompt': 'Launching a new line of eco-friendly clothing',
    },
    {
        'prompt': 'Promoting a new travel blog focusing on budget travel',
    },
    {
        'prompt': 'Advertising a new software for efficient project management',
    },
    {
        'prompt': 'Introducing a new book on mastering Python programming',
    },
    {
        'prompt': 'Promoting a new online platform for learning languages',
    },
    {
        'prompt': 'Advertising a new service for personalized meal plans',
    },
    {
        'prompt': 'Launching a new app for mental health and mindfulness',
    }
]

if use_wandb:
    wandb.config.update({"description": description,
                        "test_cases": test_cases})

In [12]:
generate_optimal_prompt(description, test_cases, NUMBER_OF_PROMPTS, use_wandb)

  0%|▏                                | 2/450 [00:04<17:30,  2.34s/it]

Draw


  1%|▏                                | 3/450 [00:08<22:39,  3.04s/it]

Draw


  1%|▎                                | 4/450 [00:13<26:14,  3.53s/it]

...

Draw


100%|███████████████████████████████| 450/450 [55:35<00:00,  3.37s/it]

Winner: **Prompt:** You are developing a landing page for a cutting-edge fitness app called "Smartly". Generate a compelling headline that will captivate potential users and convince them to explore the app further.

**What the AI should see:** A prompt describing the need for a landing page headline for the fitness app "Smartly".

**What the AI is allowed to output:** A catchy and attention-grabbing headline that effectively promotes the Smartly fitness app without mentioning specific features or examples.


100%|███████████████████████████████| 450/450 [55:39<00:00,  7.42s/it]

Draw
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|                                                                                                                                                                                                                                                                Prompt                                                                                                                                                                                              